# Bot для торговли на бирже Binance

https://github.com/binance/binance-futures-connector-python

Стратегия торговли: вход в сделку на основе предсказаний модели обученной на данных стакана.

## Импорт библиотек

In [3]:
from binance.client import Client
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging
import pandas as pd
import numpy as np
import talib
import keys
import time
from datetime import datetime
from pytz import timezone
import logging
import requests
import json
import re
pd.options.mode.chained_assignment = None # отключение дебильного предупреждения
pd.set_option('display.max_columns', None) # показывать максимальное количесвто столбцов

## Подключение к бирже

In [4]:
def connecting_to_market():
    try:
        client_alex = UMFutures(keys.api_key, keys.secret_key)
        result_alex = f'Подключение client_alex к бирже выполено.'
    except:
        result_alex = f'client_alex: ошибка при подключении к бирже.' 

    try:
        client_den = UMFutures(keys.api_key_den, keys.secret_key_den)
        result_den = f'Подключение client_den к бирже выполено.'
    except:
        result_den = f'client_den: ошибка при подключении к бирже.'

    return client_alex, result_alex, client_den, result_den
        

In [5]:
clients = connecting_to_market()
client, client_den = clients[0], clients[2]

## Задать переменные

In [6]:
SYMBOL = 'MOODENGUSDT'

## Запуск логирования

In [7]:
def start_logging(Bot_name):
    """
    Bot_name - имя бота - str
    """
    logger = logging.getLogger(Bot_name)
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler('bot.log')
    fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(fh)
    return logger

## Получение данных

### REST_API

#### Стакан

In [8]:
client.depth(SYMBOL, limit = 10)

{'lastUpdateId': 7914725010480,
 'E': 1751294280447,
 'T': 1751294280427,
 'bids': [['0.1491800', '4065'],
  ['0.1491700', '4459'],
  ['0.1491600', '9683'],
  ['0.1491500', '12977'],
  ['0.1491400', '18774'],
  ['0.1491300', '10328'],
  ['0.1491200', '29304'],
  ['0.1491100', '30522'],
  ['0.1491000', '24344'],
  ['0.1490900', '15580']],
 'asks': [['0.1491900', '2798'],
  ['0.1492000', '111'],
  ['0.1492100', '9095'],
  ['0.1492200', '7226'],
  ['0.1492300', '5546'],
  ['0.1492400', '7841'],
  ['0.1492500', '18044'],
  ['0.1492600', '23225'],
  ['0.1492700', '27002'],
  ['0.1492800', '9725']]}

In [9]:
df = pd.DataFrame(client.depth(SYMBOL, limit = 10))

In [10]:
df.head(10)

,lastUpdateId,E,T,bids,asks
0,7914725068567,1751294280968,1751294280960,"[0.1492300, 2314]","[0.1492400, 1686]"
1,7914725068567,1751294280968,1751294280960,"[0.1492200, 2052]","[0.1492500, 1815]"
2,7914725068567,1751294280968,1751294280960,"[0.1492100, 7906]","[0.1492600, 3086]"
3,7914725068567,1751294280968,1751294280960,"[0.1492000, 13918]","[0.1492700, 3499]"
4,7914725068567,1751294280968,1751294280960,"[0.1491900, 9315]","[0.1492800, 7455]"
5,7914725068567,1751294280968,1751294280960,"[0.1491800, 25786]","[0.1492900, 6749]"
6,7914725068567,1751294280968,1751294280960,"[0.1491700, 12697]","[0.1493000, 17402]"
7,7914725068567,1751294280968,1751294280960,"[0.1491600, 15254]","[0.1493100, 25096]"
8,7914725068567,1751294280968,1751294280960,"[0.1491500, 39508]","[0.1493200, 11834]"
9,7914725068567,1751294280968,1751294280960,"[0.1491400, 23429]","[0.1493300, 27161]"


### WebSockets

#### Функции для подключения

In [20]:
def start_Socket(handler):
    '''
    Запуск сокета
    '''
    my_client = UMFuturesWebsocketClient(on_message = handler)
    try:
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=20,
        id=1)
        time.sleep(5)
        logging.debug("closing ws connection")
        my_client.stop()
        
    except:
        time.sleep(2)
        logging.debug("connection restart")
        my_client.diff_book_depth(
        symbol="moodengusdt",
        speed=20,
        id=1)
        time.sleep(15000)
        logging.debug("closing ws connection")
        my_client.stop()


In [12]:
data = []

In [13]:
def handler_debug(_, message):
    """
    Вывод всей инф стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        data.append(message)
        print(message)

In [14]:
def handler_orderbook_update(_, message):
    """
    Предобработка данных стакана
    """
    if json.loads(message)['e'] == 'depthUpdate':
        try:
            bids = [float(b[1]) for b in json.loads(message)['b']][::-1][:10]
            asks = [float(a[1]) for a in json.loads(message)['a']][:10]
            mid_price = [(float(json.loads(message)['a'][-1][0]) + float(json.loads(message)['b'][::-1][0][0])) / 2] 
            total_feat = mid_price + asks + bids 
            df_row = pd.DataFrame(total_feat).T
            df_row.columns = ['mid_price', 'ask_1', 'ask_2', 'ask_3', 'ask_4', 'ask_5', 'ask_6', 'ask_7', 'ask_8', 'ask_9', 'ask_10', 
                        'bid_1', 'bid_2', 'bid_3', 'bid_4', 'bid_5', 'bid_6', 'bid_7', 'bid_8', 'bid_9', 'bid_10']
            data.append(df_row)
            #print(message)
        except:
            df_row = pd.DataFrame()  
            data.append(df_row)
            print(message)  

#### Запуск сбора данных

In [21]:
start_Socket(handler_orderbook_update)

ERROR:binance.websocket.websocket_client:Error from callback <function handler_orderbook_update at 0x000002467F453400>: 'e'


#### Получение датафрейма

In [22]:
ds_ws = pd.concat(data).reset_index(drop=True)

In [23]:
ds_ws.shape

(7, 21)

In [24]:
ds_ws.head(2)

,mid_price,ask_1,ask_2,ask_3,ask_4,ask_5,ask_6,ask_7,ask_8,ask_9,ask_10,bid_1,bid_2,bid_3,bid_4,bid_5,bid_6,bid_7,bid_8,bid_9,bid_10
0,0.150710,0.0,0.0,902.0,4176.0,4773.0,7895.0,11128.0,12746.0,29624.0,25263.0,0.0,5865.0,9496.0,9684.0,7727.0,17256.0,22130.0,25754.0,18185.0,14448.0
1,0.149645,774.0,4387.0,10051.0,16986.0,19842.0,23623.0,21411.0,27807.0,31722.0,26810.0,0.0,4868.0,2350.0,2046.0,7623.0,14614.0,15400.0,13609.0,20472.0,26055.0


#### Выгрузка датафрейма

In [25]:
ds_ws.to_csv('ds_moodeng_13.csv', index = False)